In [1]:
import json
import torch
from albumentations.pytorch import ToTensorV2
from datasets import SegDataset
from models import SegModel
from utils import mask_to_rle

### Load model from existing checkpoint

In [3]:
model = SegModel.load_from_checkpoint("checkpoints/epoch=1-val_loss=0.68-val_high_vegetation_IoU=48.62-val_mIoU=52.77.ckpt")

### Generate results for the validation set of the warmup phase

Change the `phase` and `split` accordingly

In [4]:
ds = SegDataset(phase="warmup", split="valid", transform=ToTensorV2())

### Convert predictions from binary masks to RLE encoding for submission

In [6]:
model.eval()
results = {}

for i, batch in enumerate(ds):
    filename = ds.img_list[i]
    img, _ = batch
    out = model(img.float().unsqueeze(dim=0).to(model.device))['out']
    probs = torch.softmax(out, dim=1)
    pred = torch.argmax(probs, dim=1)
    pred = pred.detach().cpu().numpy().squeeze()

    rle = mask_to_rle(pred)
    results[filename] = {
        "counts": rle,
        "height": pred.shape[0],
        "width": pred.shape[1],
    }

with open("results.json", "w") as f:
        json.dump(results, f)

### Zip the file `results.json`

In [7]:
!zip results.zip results.json

  adding: results.json (deflated 64%)


### Done!, you can now submit the file `results.zip`, good luck!!!